### Langchain and Pinecode 
Index = rag-idx
model = HuggingFace ('google/flan-t5-large)

In [ ]:
%pip install langchain-pinecone pinecone-client langchainhub ctransformers transformers

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader("./text_data/mediumblog1.txt")
document = loader.load()
spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20) 
texts = spliter.split_documents(document)

print (len(texts))

18


In [4]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv()
fn_embeddings = HuggingFaceEmbeddings()

PineconeVectorStore.from_documents(texts, 
                                   embedding=fn_embeddings,
                                   index_name=os.environ["PINECONE_INDEX_NAME"]
                                   )

In [7]:
from langchain_huggingface.llms import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 100},
)

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain import hub

load_dotenv()
fn_embeddings = HuggingFaceEmbeddings()

query = "what is Semantic search?"

vectorstore = PineconeVectorStore(
    index_name=os.environ["PINECONE_INDEX_NAME"],
    embedding=fn_embeddings
)
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

retrieval_chain = create_retrieval_chain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=combine_docs_chain 
)

result = retrieval_chain.invoke(input={"input":query})


In [ ]:
from langchain_core.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know, just say you don't know, don't try to make up an answer.
Use 3 sentences maximum and keep the answer concise as possible. Every sentence fits in a line.
Always say "Thanks for asking!" at the end of answer.
{context}
Question: {question}
Helpful answer:
"""

custom_rag_prompt = PromptTemplate.from_template(template=template)